In [1]:
!pip3 install yfinance
!pip3 install mftool

In [13]:
import numpy as np 
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import datetime
import yfinance as yfin
yfin.pdr_override()

from mftool import Mftool

import locale
curr = locale.getdefaultlocale()
locale.setlocale(locale.LC_ALL, curr)

def get_pf_perf(day):
    
    mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/myTransactions.csv',encoding= 'unicode_escape')

    stocks = mypf[mypf['type'] != 'mf']['id'].unique()

    stock_df = pd.DataFrame()

    for stock in stocks:
        #stock = 'alkylamine'
        stock_id = stock.upper() + '.NS'
        start_date = (datetime.date.today() + datetime.timedelta(days=-5)).strftime('%Y-%m-%d')    
        end_date = (datetime.date.today() + datetime.timedelta(days=0)).strftime('%Y-%m-%d')
        tmp_df = round(web.DataReader(stock_id,  data_source='yahoo', start = start_date, end = end_date)['Close'],2)
        tmp_df = pd.DataFrame(tmp_df)     
        tmp_df['id'] = stock
        tmp_df.columns = ['close','id']
        tmp_df = tmp_df.dropna()
        tmp_df = tmp_df.tail(1+abs(day))
        tmp_df = tmp_df.head(1)
        stock_df = stock_df.append(tmp_df)   

    stock_df = pd.merge(mypf, stock_df,on='id')

    obj = Mftool()

    data = obj.get_scheme_codes() 

    #scrips_dict = {'135800':'LMC-Tata','122639':'IFC-PP','125354':'SC-Axis','145110':'LMC-Axis'}
    mfs = mypf[mypf['type'] == 'mf']['id'].unique()
    start_date = (datetime.date.today() + datetime.timedelta(days=-5)).strftime('%d-%m-%Y')    
    end_date = (datetime.date.today() + datetime.timedelta(days=0)).strftime('%d-%m-%Y')

    df_mfnav = pd.DataFrame()

    for key in mfs:
        obj.get_scheme_details(key)
        tmp = obj.get_scheme_historical_nav_for_dates(key, start_date, end_date)
        tmp = pd.DataFrame(tmp['data'])
        tmp.columns = ['date', 'close']
        tmp['id'] = key
        cols = ['close','id']
        tmp = tmp[cols]
        tmp = tmp.tail(1+abs(day))
        tmp = tmp.head(1)
        df_mfnav = df_mfnav.append(tmp)

    df_mfnav['close'] = pd.to_numeric(df_mfnav['close'])
    mf_df = pd.merge(mypf, df_mfnav,on='id')
    mf_df['close'] = mf_df['close'].astype(np.float64)
    mypf = stock_df.append(mf_df)

    mypf['cost'] = mypf['quantity']*mypf['price']
    mypf['value'] = mypf['quantity']*mypf['close']
    pf_cost = round(sum(mypf['cost']),2)
    pf_revenue = round(sum(mypf['value']),2)

    stock_df = mypf[mypf['type'] != 'mf']
    stock_cost = round(sum(stock_df['cost']),2)
    stock_revenue = round(sum(stock_df['value']),2)

    mf_df = mypf[mypf['type'] == 'mf']
    mf_cost = round(sum(mf_df['cost']),2)
    mf_revenue = round(sum(mf_df['value']),2)
    
    return pf_cost, pf_revenue, stock_cost, stock_revenue, mf_cost, mf_revenue

day = 0
pf_cost, pf_revenue, stock_cost, stock_revenue, mf_cost, mf_revenue = get_pf_perf(day)
prv_pf_cost, prv_pf_revenue, prv_stock_cost, prv_stock_revenue, prv_mf_cost, prv_mf_revenue = get_pf_perf(day-1)

pf_profit = round(pf_revenue - pf_cost,2)
pf_profit_perc = round(pf_profit/pf_cost,4)

pf_growth_yday = round(pf_revenue - prv_pf_revenue,2)
pf_growth_yday_perc = round(pf_growth_yday/prv_pf_revenue,4)

pf_nav = round(pf_revenue*10/pf_cost,2)

stock_profit = round(stock_revenue - stock_cost,2)
stock_profit_perc = round(stock_profit/stock_cost,4)

stock_growth_yday = round(stock_revenue - prv_stock_revenue,2)
stock_growth_yday_perc = round(stock_growth_yday/prv_stock_revenue,4)

mf_profit = round(mf_revenue - mf_cost,2)
mf_profit_perc = round(mf_profit/mf_cost,4)

mf_growth_yday = round(mf_revenue - prv_mf_revenue,2)
mf_growth_yday_perc = round(mf_growth_yday/prv_mf_revenue,4)

print('==Portfolio==')
print("Portfolio Cost : " +(locale.currency(pf_cost, grouping=True)))
print("Portfolio Revenue : " +(locale.currency(pf_revenue, grouping=True)))
print('------')
print("Portfolio Profit : " +(locale.currency(pf_profit, grouping=True)))
print("Portfolio Profit Perc : {:2.2%}".format(pf_profit_perc))
print('------')
print("Portfolio Day Growth : " +(locale.currency(pf_growth_yday, grouping=True)))
print("Portfolio Day Growth Perc : {:2.2%}".format(pf_growth_yday_perc))
print('------')
print("PF NAV : " +(locale.currency(pf_nav, grouping=True)))

print('==Stocks==')
print("Stocks Cost : " +(locale.currency(stock_cost, grouping=True)))
print("Stocks Revenue : " +(locale.currency(stock_revenue, grouping=True)))
print('------')
print("Stocks Profit : " +(locale.currency(stock_profit, grouping=True)))
print("Stocks Profit Perc : {:2.2%}".format(stock_profit_perc))
print('------')
print("Stocks Day Growth : " +(locale.currency(stock_growth_yday, grouping=True)))
print("Stocks Day Growth Perc : {:2.2%}".format(stock_growth_yday_perc))

print('==MFs==')
print("MFs Cost : " +(locale.currency(mf_cost, grouping=True)))
print("MFs Revenue : " +(locale.currency(mf_revenue, grouping=True)))
print('------')
print("MFs Profit : " +(locale.currency(mf_profit, grouping=True)))
print("MFs Profit Perc : {:2.2%}".format(mf_profit_perc))
print('------')
print("MFs Day Growth : " +(locale.currency(mf_growth_yday, grouping=True)))
print("MFs Day Growth Perc : {:2.2%}".format(mf_growth_yday_perc))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********